In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
# preprocessing starts here
lemmatizer= WordNetLemmatizer()

In [ ]:
intents= json.loads(open('/content/intents.json').read())
words=[]
classes=[]
documents= []
ignore_letters=['?','!','.',',']

In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [ ]:
words= [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words= sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training= []
output_empty = [0]*len(classes)

In [ ]:
for document in documents:
  bag= []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)

<ipython-input-46-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# segregating dependent and independent variables
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation, Dropout
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
# Building the model
model_finale= Sequential()
model_finale.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model_finale.add(Dropout(0.5))
model_finale.add(Dense(64, activation='relu'))
model_finale.add(Dropout(0.5))
model_finale.add(Dense(len(train_y[0]), activation = 'softmax'))

# sgd = SGD( decay= 1e-6, momentum=0.9, nesterov=True)

model_finale.compile(loss= 'categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])

In [ ]:
hist2 = model_finale.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
12/12 [==============================] - 2s 7ms/step - loss: 1.8162 - accuracy: 0.2167
Epoch 2/200
12/12 [==============================] - 0s 6ms/step - loss: 1.7515 - accuracy: 0.2833
Epoch 3/200
12/12 [==============================] - 0s 7ms/step - loss: 1.7044 - accuracy: 0.3000
Epoch 4/200
12/12 [==============================] - 0s 6ms/step - loss: 1.6097 - accuracy: 0.4000
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 1.5767 - accuracy: 0.4500
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 1.5775 - accuracy: 0.4333
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 1.5379 - accuracy: 0.4500
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 1.4546 - accuracy: 0.5833
Epoch 9/200
12/12 [==============================] - 0s 3ms/step - loss: 1.3919 - accuracy: 0.5667
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 1.3381 - accuracy: 0.7500
Epoch 11/

In [ ]:
model_finale.save('chatbot_model.h5', hist2)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
